In [1]:
%run load_data.py
import sys
import multiprocessing as mp
import numpy as np
import scipy.sparse.linalg as splin
import scipy.sparse as sparse
import random
import math

np.random.seed(42)
random.seed(42)

In this document the methods described in the methods section are implemented for the network of verdicts. The data is loaded as `networkx`directed graph making it relatively easy to work with. The goal is to set up an easily used interface for running K-folds cross validation on the network for different link prediction algorithms and evaluate them with ROC and precision.

In [2]:
# Find the greatest connected component and work on that
components = []
lengths = []
# Find the greatest component from the undirected version of the graph
for component in nx.connected_component_subgraphs(nx.Graph(G)):
    components.append(component)
    lengths.append(len(component))
# Find the GCC as the largest component and then recreate the directed graph
GCC = components[lengths.index(max(lengths))]
GCC = G.subgraph(GCC.nodes())

Since this is an undirected implementation we'll need an undirected version of the GCC. Changing to undirected has a tendency to swap edge the position nodes on edges. Position is necessary to trim the graph, so we'll make sure that `edge[0]` is always younger than `edge[1]`.

In [3]:
# Create an undirected copy of the GCC and make sure that the edges in the copy respect causality
GCC_un = nx.Graph(GCC.copy())

In [11]:
def jaccard(validation_set, G):
    """
    Perform Jaccard scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_un = 1.0*len(u.union(v))
        uv_int = 1.0*len(u.intersection(v))
        if uv_int == 0 or uv_un == 0:
            s= 0.0
        else:
            s = uv_int/uv_un
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results
    
def common_neighbors(validation_set, G):
    """
    Perform common neighbors scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        s = len(u.intersection(v))
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def adamic_adar(validation_set, G):
    """
    Perform Adamic/Adar scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_int = u.intersection(v)
        s = sum([1.0/math.log(G.degree(node)) for node in uv_int if G.degree(node) != 1 and G.degree(node) != 0])
        if s == None:
            s = 0.0
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def resource_allocation(validation_set, G):
    """
    Perform resource allocation scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_int = u.intersection(v)
        s = sum([1.0/G.degree(node) for node in uv_int if G.degree(node) != 0])
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def leicht_holme_newman(validation_set, G):
    """
    Perform LHN1 scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_int = 1.0*len(u.intersection(v))
        if G.degree(non_edge[0]) == 0 or G.degree(non_edge[1]) == 0:
            s = 0.0
        else:
            s = uv_int/(G.degree(non_edge[0])*G.degree(non_edge[1]))
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def sorensen(validation_set, G):
    """
    Perform Sørensen index scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_int = 1.0*len(u.intersection(v))
        if G.degree(non_edge[0]) == 0 and G.degree(non_edge[1]) == 0:
            s = 0.0
        else:
            s = 2.0*uv_int/(G.degree(non_edge[0])+G.degree(non_edge[1]))
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def hub_promoted(validation_set, G):
    """
    Perform hub promoted index scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_int = 1.0*len(u.intersection(v))
        if G.degree(non_edge[0]) == 0 and G.degree(non_edge[1]) == 0:
            s = 0.0
        else:
            s = uv_int/max(G.degree(non_edge[0]), G.degree(non_edge[1]))
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def hub_depressed(validation_set, G):
    """
    Perform hub promoted index scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_int = 1.0*len(u.intersection(v))
        if G.degree(non_edge[0]) == 0 or G.degree(non_edge[1]) == 0:
            s = 0.0
        else:
            s = uv_int/min(G.degree(non_edge[0]), G.degree(non_edge[1]))
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def salton(validation_set, G):
    """
    Perform Salton index scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        u = set(G.neighbors(non_edge[0]))
        v = set(G.neighbors(non_edge[1]))
        uv_int = 1.0*len(u.intersection(v))
        if G.degree(non_edge[0]) == 0 or G.degree(non_edge[1]) == 0:
            s = 0.0
        else:
            s = uv_int/math.sqrt(G.degree(non_edge[0])*G.degree(non_edge[1]))
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

def preferential_attachment(validation_set, G):
    """
    Perform preferential attachment scoring on a single edge
    
    arguments:
    non_edge -- edge tuple specified by node endpoints
    G -- graph containing the nodes in the edge
    
    return:
    edges with the score as an attribute
    """
    results = []
    for non_edge in validation_set:
        s = G.degree(non_edge[0])*G.degree(non_edge[1])
        non_edge[2]['score'] = s
        results.append(non_edge)
    return results

## Global index similarities, computed based on every path between two nodes. SLower than local indices but contain full
## information
def katz_index(validation_set, G, beta):
    """
    Return a weighted sum over every path between two nodes, s_xy = sum(beta*A+beta^2*A^2+...) = (I-beta*A^(-1))-I
    
    arguments:
    validation_set -- set of edges between nodes to be considered
    A_inv -- Precomputed inverse of the adjacency matrix
    beta -- weighting parameter, the smaller it is the closer the result is to CN
    
    return:
    list of edges with the score as an attribute
    """
    A = nx.adjacency_matrix(G)
    # Create indices to access A
    id_to_ind = {node: i for (i, node) in zip(range(0, len(G.nodes())), G.nodes())}
    max_eig = splin.eigsh(A.asfptype(), k=1, which='LM', return_eigenvectors=False)[0]
    if 1.0/max_eig <= beta:
        raise Exception("Beta must be less than or equal to the maximum eigenvalue of A(G), which is: {}".format(1.0/max_eig))
    
    I = np.identity(A.shape[0])
    S = np.linalg.inv(I-beta*A.todense()) - I
    results = [(x,y, {'score': S[id_to_ind[x], id_to_ind[y]]}) for (x,y) in validation_set]
    return results

def leicht_holme_newman_global(validation_set, G, alpha):
    """
    Return a weighted sum of every path between two nodes, s_xy = sum(I+omega*A+omega^2*A^2+...)
    """
    A = nx.adjacency_matrix(G)
    # Create indices to access A
    id_to_ind = {node: i for (i, node) in zip(range(0, len(G.nodes())), G.nodes())}
    M = len(G.edges())
    max_eig = splin.eigsh(A.asfptype(), k=1, which='LM', return_eigenvectors=False)[0]
    I = np.identity(A.shape[0])
    D_inv = np.linalg.inv(I+np.diag(G.degree().values())) # TODO: This normally does not have the I factor, but this is done to remove 0-rows creating singular matrices
    A_inv=np.linalg.inv((I-alpha*A)/max_eig)
    S = 2*M*max_eig*D_inv*A_inv*D_inv
    results = [(x,y, {'score': S[id_to_ind[x], id_to_ind[y]]}) for (x,y) in validation_set]
    return results

def average_commute_time(validation_set, G):
    """
    Return the score as the inverse of the average commute time between two nodes, found as the pseudo inverse of the Laplacian
    """
    A = nx.adjacency_matrix(G)
    # Create indices to access A
    id_to_ind = {node: i for (i, node) in zip(range(0, len(G.nodes())), G.nodes())}
    L_plus = np.linalg.pinv(nx.laplacian_matrix(G).todense())
    results = []
    scoring = lambda x,y: 1.0/(L_plus[id_to_ind[x], id_to_ind[x]] + L_plus[id_to_ind[y], id_to_ind[y]] - 2*L_plus[id_to_ind[x], id_to_ind[y]])
    for x,y in validation_set:
        results.append((x,y, {'score': scoring(x,y)}))
    return results

def local_path_index(validation_set, G, epsilon, n):
    """
    The same as the katz index, but constricted to paths of length n
    """
    results = []
    for x,y in validation_set:
        count = [0 for _ in range(0, n+1)]
        q = [(x, 0)]
        while len(q) != 0:
            v, dist = q.pop()
            for w in G.neighbors(v):
                if dist < (n-1):
                    q.append((w, dist + 1))
                if dist < n and w == y:
                    count[dist] += 1
        s = sum([math.pow(epsilon,1.0*i)*c for i, c in zip(range(0, len(count) + 1), count)])
        results.append((x,y, {'score': s}))
    return results

def local_path_index2(validation_set, G, epsilon, n):
    """
    The same as the katz index, but constricted to paths of length n
    """
    results = []
    for x,y in validation_set:
        s = 0.0
        neighborhood = set(G.neighbors(x))
        for i in range(2, n+2):
            if y in neighborhood:
                H = G.subgraph(neighborhood)
                id_to_ind = {node: i for (i, node) in zip(range(0, len(H.nodes())), H.nodes())}
                A = np.linalg.matrix_power(nx.adjacency_matrix(H).todense(), i)
                s += math.pow(epsilon, i-2)*A[id_to_ind[x], id_to_ind[y]]
            
            for neighbor in list(neighborhood):
                neighborhood.update(set(G.neighbors(neighbor)))
        results.append((x,y,{'score': s}))
    return results

def local_random_walk(validation_set, G, t):
    """
    
    arguments:
    validation_set -- set of edges between nodes to be considered
    G -- the graph the validation set is evaluated against
    n -- number of steps to be considered
    
    return:
    list of edges with the score as an attribute
    """
    for x,y in validation_set:
        pi_xy = 0
        pi_yx = 0
        for tau in range(1, t+1):
            d_x = 0

def non_edges(graph):
    """
    Returns the non-existent edges in the graph in a randomized order.
    Monkeypatch of the original networkx implementation and will implode under strain due to not using iterators

    Parameters
    ----------
    graph : NetworkX graph.
        Graph to find non-existent edges.

    Returns
    -------
    non_edges : iterator
        Iterator of edges that are not in the graph.
    """
    if graph.is_directed():
        nodes = graph.nodes()
        random.shuffle(nodes)
        for u in nodes:
            for v in nx.non_neighbors(graph, u):
                yield (u, v)
    else:
        nodes = list(set(graph))
        random.shuffle(nodes)
        nodes = set(nodes)
        while nodes:
            u = nodes.pop()
            for v in nodes - set(graph[u]):
                yield (u, v)
                
def valid_random_non_edges(graph, n):
    """
    Returns randomized, non-existent links between nodes in the graph that are guaranteed to observe causality.

    Parameters
    ----------
    graph : NetworkX graph.
        Graph to find non-existent edges.

    Returns
    -------
    non_edges : iterator
        Iterator of edges that are not in the graph.
    """
    result_pairs = []
    # Sort edges according to age
    sorted_edges =[node for node, data in sorted(graph.nodes(data=True), key=lambda x: x[1]['date'], reverse=True)]
    node_set = set(graph.nodes())
    candidates = list(np.random.choice(sorted_edges, n, replace=True))
    i = 0
    while i < len(candidates):
        u = candidates[i]
        # Make sure the potential neighbors respect causality with a resolution equal to the timestamp
        cand_index = sorted_edges.index(u)
        potential_neighbors = set(sorted_edges[cand_index:])
        if graph.is_directed():
            neighbors = set(graph.successors(u)).union(set(graph.predecessors(u)))
        else:
            neighbors = set(graph.neighbors(u))
        # Make sure the potential neighbors respect causality
        non_neighbors = list(potential_neighbors - neighbors)
        # The oldest node will have a neighborhood of Ø, so add a new candidate to the list in that case
        if len(non_neighbors) == 0:
            candidates.append(random.choice(graph.nodes()))
        else:    
            result_pairs.append((u, random.choice(non_neighbors)))
        i += 1
    return result_pairs
        

def k_fold_validate(G, k, fun, **kwargs):
    """
    K-fold validation of some specified function
    
    arguments:
    G -- Graph to perform the function on
    k -- number of folds
    fun -- function to be evaluated
    kwargs -- arguments to be passed to the evaluated function
    
    return:
    List of lists of scored predictions
    """
    # Use an undirected graph for now
    G_undirected = nx.Graph(G.copy())
    edges = G_undirected.edges(data=True)
    
    random.shuffle(edges)
    # Find the number of true members in the validation set
    N = len(edges)/k
    validation_sets = []
    for i in range(0,k):
        validation_sets.append(edges[i*N:(i+1)*N])
    results = []
    for true_validation_set in validation_sets:
        # Create a training set and remove all true members of the validation set from it
        G_train = G_undirected.copy()
        G_train.remove_edges_from(true_validation_set)
        
        # Fetch random edges guaranteed not to be in the graph
        false_validation_set = valid_random_non_edges(G_undirected, len(true_validation_set))
        # Trim out and replace edges if they break causality. Continue doing this until the set is the size of the validation set
        for i in range(len(false_validation_set)):
            edge = false_validation_set[i]
            # Add date information to the non-edge
            false_validation_set[i] = edge + ({'date': G.node[edge[0]]['date']},)
        
        # Make sure the true edges have sane orientation since this can be flipped when creating an undirected graph
        for i in range(len(true_validation_set)):
            edge = true_validation_set[i]
            if G.node[edge[0]]['date'] < G.node[edge[1]]['date']:
                true_validation_set[i] = (edge[1], edge[0], edge[2])
        
        validation_set = true_validation_set + false_validation_set
        results.append(fun(validation_set, G_train, **kwargs))
    return results

def precision(G, results, L):
    """
    Find the ratio of the true positives to trues
    
    arguments:
    G -- graph the results are based on
    results -- list of lists of scored predictions
    L -- number of results to be considered
    
    return:
    List of precisions for each set of results
    """
    # Sort the results with descending scores
    results = [sorted(result, key=lambda x: x[2]['score'], reverse=True) for result in results]
    edge_set = set(G.edges())
    # True positives exist in both the edge set and the result set
    true_positives = [[(edge[0],edge[1]) for edge in result[0:L] if (edge[0], edge[1]) in edge_set] for result in results]
    return [1.0*len(trues)/L for trues in true_positives]

def AUC(G, results):
    """
    Perform n trials where the score of a non-edge and an edge in the result is compared. Count the number of trials where
    the edge had the higher score as n' and the number of times the score was equal as n'' and return the AUC as (n' + n'')/n.
    
    arguments:
    G -- graph the results are based on
    results -- list of lists of scored predictions
    
    return:
    List of precisions for each set of results
    """
    
    edge_set = set(G.edges())
    AUC = []
    for result_set in results:
        true_edges = []
        false_edges = []
        for (x,y,data) in result_set:
            if (x,y) in edge_set:
                true_edges.append((x,y,data))
            else:
                false_edges.append((x,y,data))
        
        random.shuffle(true_edges)
        random.shuffle(false_edges)
        n = len(true_edges)
        n_better = 0.0
        n_same = 0.0
        for i in range(0, n):
            if true_edges[i][2]['score'] > false_edges[i][2]['score']:
                n_better += 1.0
            if true_edges[i][2]['score'] == false_edges[i][2]['score']:
                n_same += 1.0
        AUC.append((n_better + 0.5*n_same)/n)
    return AUC
        

In [16]:
false_validation_set = valid_random_non_edges(GCC_un, 8733)
for i in range(len(false_validation_set)):
    edge = false_validation_set[i]
    # Add date information to the non-edge
    false_validation_set[i] = edge + ({'date': GCC_un.node[edge[0]]['date']},)

In [20]:
aa = adamic_adar(false_validation_set, GCC_un)

In [28]:
sorted(aa, key=lambda x: x[2]['score'], reverse=True)

[('61974CJ0033',
  '61974CJ0033',
  {'date': datetime.date(1974, 12, 3), 'score': 7.761331334276768}),
 ('61983CJ0246',
  '61983CJ0246',
  {'date': datetime.date(1985, 5, 2), 'score': 6.998039195722067}),
 ('62011CJ0201',
  '62011CJ0201',
  {'date': datetime.date(2013, 7, 18), 'score': 4.645992227886982}),
 ('61954CJ0002',
  '61954CJ0002',
  {'date': datetime.date(1954, 12, 21), 'score': 4.596541945290206}),
 ('61954CJ0002',
  '61954CJ0002',
  {'date': datetime.date(1954, 12, 21), 'score': 4.596541945290206}),
 ('61954CJ0002',
  '61954CJ0002',
  {'date': datetime.date(1954, 12, 21), 'score': 4.596541945290206}),
 ('61954CJ0001',
  '61954CJ0001',
  {'date': datetime.date(1954, 12, 21), 'score': 4.352990423095306}),
 ('61955CJ0008',
  '61955CJ0008',
  {'date': datetime.date(1956, 7, 16), 'score': 3.2362622188606336}),
 ('61959CJ0046',
  '61959CJ0046',
  {'date': datetime.date(1962, 12, 14), 'score': 2.7659903543670996}),
 ('61980CJ0269',
  '61980CJ0269',
  {'date': datetime.date(1981, 12

In [33]:
len(filter(lambda x: x[2]['score'] != 0.0, aa))

120

In [35]:
edges = GCC_un.edges(data=True)
random.shuffle(edges)
true_validation_set = edges[0:8733]

In [36]:
aa2 = adamic_adar(true_validation_set, GCC_un)
len(filter(lambda x: x[2]['score'] != 0.0, aa2))

7012

In [19]:
false_validation_set[0][2]

{'date': datetime.date(2009, 7, 16)}

IndexError: tuple index out of range

In [ ]:
sorted_edges =[n for n, data in sorted(GCC.nodes(data=True), key=lambda x: x[1]['date'], reverse=True)]

In [ ]:
np.random.choice(sorted_edges, 1, replace=True)

In [ ]:
N = len(GCC_un.edges())/5

In [ ]:
N

In [ ]:
%time acm_results = k_fold_validate(GCC, 5, average_commute_time)

In [ ]:
%time lhn2_results = k_fold_validate(GCC, 5, leicht_holme_newman_global, alpha=.95)

In [ ]:
%time lp_results = k_fold_validate(GCC, 5, local_path_index, epsilon=0.01, n=3)

In [ ]:
print "LP:"
print sum(precision(GCC, lp_results, 50))/5
print sum(AUC(GCC, lp_results))/5
print "ACM:"
print sum(precision(GCC, acm_results, 50))/5
print sum(AUC(GCC, acm_results))/5
print "LHN2:"
print sum(precision(GCC, lhn2_results, 50))/5
print sum(AUC(GCC, lhn2_results))/5

In [ ]:
%time katz_results = k_fold_validate(GCC, 5, katz_index, beta=0.04)

In [ ]:
print "Katz:"
print sum(precision(GCC, katz_results, 50))/5
print sum(AUC(GCC, katz_results))/5

In [ ]:
cn_results = k_fold_validate(GCC, 5, common_neighbors)
jaccard_results = k_fold_validate(GCC, 5, jaccard)
salton_results = k_fold_validate(GCC, 5, salton)
pa_results = k_fold_validate(GCC, 5, preferential_attachment)
hdi_results = k_fold_validate(GCC, 5, hub_depressed)
hpi_results = k_fold_validate(GCC, 5, hub_promoted)
ra_results = k_fold_validate(GCC, 5, resource_allocation)
aa_results = k_fold_validate(GCC, 5, adamic_adar)
sorensen_results = k_fold_validate(GCC, 5, sorensen)
lhn_results = k_fold_validate(GCC,5, leicht_holme_newman)

In [ ]:
print "CN:"
print sum(precision(GCC, cn_results, 150))/5
print sum(AUC(GCC, cn_results))/5
print "Jaccard:"
print sum(precision(GCC, jaccard_results, 150))/5
print sum(AUC(GCC, jaccard_results))/5
print "Salton:"
print sum(precision(GCC, salton_results, 150))/5
print sum(AUC(GCC, salton_results))/5
print "Preferential Attachment:"
print sum(precision(GCC, pa_results, 150))/5
print sum(AUC(GCC, pa_results))/5
print "Hub depressed:"
print sum(precision(GCC, hdi_results, 150))/5
print sum(AUC(GCC, hdi_results))/5
print "Hub promoted:"
print sum(precision(GCC, hpi_results, 150))/5
print sum(AUC(GCC, hpi_results))/5
print "Resource allocation:"
print sum(precision(GCC, ra_results, 150))/5
print sum(AUC(GCC, ra_results))/5
print "Adamic/Adar:"
print sum(precision(GCC, aa_results, 150))/5
print sum(AUC(GCC, aa_results))/5
print "Sorensen:"
print sum(precision(GCC, sorensen_results, 150))/5
print sum(AUC(GCC, sorensen_results))/5
print "LHN:"
print sum(precision(GCC, lhn_results, 150))/5
print sum(AUC(GCC, lhn_results))/5

In [12]:
%time aa_results = k_fold_validate(GCC, 5, adamic_adar)

Wall time: 52.6 s


In [13]:
print "Adamic/Adar:" 
print sum(precision(GCC, aa_results, 50))/5
print sum(AUC(GCC, aa_results))/5

Adamic/Adar:
0.964
0.843418238535
